In [ ]:
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path
import pandas as pd
import numpy as np
from pymongo import MongoClient
from datetime import datetime, timedelta
import duckdb as ddb
import itertools as it

In [ ]:
experiment_path='G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\Spindlewachstumsversuch\\220429_03\\'

In [ ]:
###Database connection configuratuions
host= "localhost"
port=27017
hf_selected_columns= ['ENC1_POS|8','is_speed',  'is_acceleration', 'POWER|8',  'CURRENT|8',  'ENC2_POS|3']

In [ ]:
###1-Load HF-Data from Database and combine them in one data frame as follows: CMD_SPEED|8  POWER|8  CURRENT|8  ENC1_POS|8  ENC2_POS|3
client = MongoClient(host= host, port=port)
db = client.h4s
event_list = db.ipmmeasurements.find()
hf_signals=None
hf_date=None
resolution= 1
for event in event_list:
    hf_date= event['date']
    resolution= event['resolution']
    signal= pd.DataFrame(event['content'])
    if hf_signals is None:
        hf_signals= signal
    else:
        hf_signals=hf_signals.join(signal)
# Generate date field for all data records based on the constant sampling rate
size=hf_signals.shape[0]    
dates= [ hf_date + timedelta(seconds= i * resolution) for i in range(size)]
hf_signals['date']= dates


In [ ]:
### Add new derived signals (Speed, accelaration)
hf_signals['is_speed']= np.gradient(hf_signals['ENC1_POS|8'])/(120/10000)
hf_signals['is_acceleration']= np.gradient(hf_signals['is_speed'])
#hf_signals['CMD_SPEED|8']=  hf_signals['CMD_SPEED|8']/6
hf_signals['ENC2_POS|3']= hf_signals['ENC2_POS|3'].apply(np.round,args=(2,))

In [ ]:
####Visualization of signals:
selected_columns= hf_selected_columns
y_axis_names= ['ENC1_POS|8 (degee)     ', 'is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)']
fig= make_subplots(rows=len(selected_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
for i in range(len(selected_columns)):
    fig.add_trace(go.Line(y= hf_signals[selected_columns[i]]), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
fig.update_layout(height=1200, width=1200, title_text="HF-Data")
fig.show()

In [ ]:
###Set synchronization point of HF-Data
sync_point_hf= 0
hf_signals= hf_signals[sync_point_hf:]

In [ ]:
# Read Laser Data
p = Path(experiment_path)
file_list = list(p.glob('*.csv'))
laser_df= pd.DataFrame()
for file in file_list:
    print('Current File: ', file)
    laser_df= pd.read_csv(file, sep=";",skiprows=range(0,7), names=['date_time','distance','nothing'], skipinitialspace= True)
print(laser_df.shape)

In [ ]:
# Filter laser data and calcuate median distance for visualization
#laser_df['distance'].mask(laser_df['distance'] < 0, 0.010, inplace= True)
laser_df['median_distance']= laser_df['distance'].rolling(20).median()

In [ ]:
laser_df

In [ ]:
######Set synchronization point of laser-Data and get the relevant segment
sync_point_laser= 14723+ 10000
f= 10000
seconds= 20
segment= laser_df[sync_point_laser: -50000]
segment.reset_index(inplace= True) 
segment.drop(columns= [ 'nothing', 'index'],inplace=True)#'date_time',

In [ ]:
segment['displacement']= (segment['median_distance'][0] - segment['median_distance'])#*10000
#result_df['displacement']= result_df['displacement'].mask(result_df['displacement']<0, 0)
segment['rolling_mean_displacement']= segment['displacement'].rolling(200).mean()

In [ ]:
#segment= laser_df
selected_columns= ['displacement']#['rolling_mean_displacement']
y_axis_names= selected_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
fig= make_subplots(rows=len(selected_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
for i in range(len(selected_columns)):
    fig.add_trace(go.Line(y= 1000 * segment[selected_columns[i]]), row= i+1, col= 1)
    fig.update_yaxes(title_text= str(y_axis_names[i] + 'Micro-Meter'), row= i+1, col= 1)
fig.update_layout(height=600, width=1200, title_text="Laser-Data_ F= 4000 Hz")
fig.show()

In [ ]:
def calc_fft(signal_data, n, sampling_rate):
    signal_data_std= signal_data #- np.mean(signal_data)
    data_fft= fft(np.asarray(signal_data_std))
    num_samples= len(signal_data)
    power = np.abs(data_fft[:len(data_fft)])
    freq_axis= fftfreq(n, sampling_rate)
    return freq_axis, power

In [ ]:
# ### End-based synchronization
# a= 516819
# b= 600100
# shift= b-a
# segment= segment[shift:]
# segment.reset_index(inplace= True)


In [ ]:
##Check the cutting point
#segment= laser_df['median_distance'][0:100000]

In [ ]:
# fig= ex.line(segment,y='median_distance')
# fig.update_layout(height=1200, width=1200, title_text="Laser-Data")
# fig.show()

In [ ]:
# fig= ex.line(segment,y='rolling_std')
# fig.update_layout(height=1200, width=1200, title_text="Laser-Data")
# fig.show()

In [ ]:
#rule= ('property', value)
def cut_signal(df: pd.DataFrame, rule:tuple):
    property= rule[0]
    value= rule[1]
    result= None
    for index,row in df.iterrows():
        if float(row[property]) > value:
            print('index= ', index, ', value= ',row[property])
            result=df.iloc[index:]
            break
    result.reset_index(inplace= True)
    return result

In [ ]:
#rule= {'df1':('is_speed',0), 'df2':('DRZ',0)}
def synchronize(df1:pd.DataFrame, df2: pd.DataFrame, rule: dict=None, resolution= 0.002):
    hf_df= df1
    lf_df= df2
    print('hf_df.shape: ',hf_df.shape)
    print('lf_df.shape: ', lf_df.shape)
    if rule is not None:
        hf_df= cut_signal(df1, rule['df1'])
        lf_df= cut_signal(df2, rule['df2'])
    date= lf_df.iloc[0]['date']
    size=hf_df.shape[0]    
    dates= [ date + timedelta(seconds= i * resolution) for i in range(size)]
    hf_df['date']= dates
    hf_df_result= pd.DataFrame()
    for index, row in lf_df.iterrows():
        lf_date= row['date']
        nearest_record= hf_df[hf_df['date']<=lf_date].sort_values('date', ascending= False, inplace= False).iloc[[0]]
        hf_df_result= pd.concat([hf_df_result, nearest_record], axis=0, ignore_index= True)
        #print(hf_df.shape)
        #print('############################')
    result= pd.concat([hf_df_result,lf_df],  axis=1)
    result.drop(columns=['index'], inplace= True)
    result.reset_index(inplace= True)
    print(result.shape)
    return result


In [ ]:
#rule= {'df1':('is_speed',0), 'df2':('DRZ',0)}
def synchronize2(df1:pd.DataFrame, df2: pd.DataFrame, rule: dict=None, hf_resolution= 0.00001, lf_resolution= 0.002):
    hf_df= df1.reset_index(inplace= False)
    lf_df= df2.reset_index(inplace= False)
    print('hf_df.shape: ',hf_df.shape)
    print('lf_df.shape: ', lf_df.shape)
    conversion_rate= int(lf_resolution/hf_resolution)
    hf_df = hf_df.groupby(np.arange(len(hf_df))//conversion_rate).median()
    len_lf_df= len(lf_df)
    len_hf_df= len(hf_df)
    if (len_hf_df> len_lf_df):
        hf_df= hf_df[:len_lf_df]
    elif (len_lf_df > len_hf_df ):
        lf_df= lf_df[:len_hf_df]    
    print(hf_df.info())
    print(lf_df.info())
    result= pd.concat([hf_df,lf_df],  axis=1)
    result.drop(columns=['index'], inplace= True)
    result.reset_index(inplace= True)
    print(result.info())
    return result

In [ ]:
##Synchronize HF and laser data HF based on Z axis, laser based on the point of ramp up
hf_laser_sync= synchronize2(segment['distance'],hf_signals, hf_resolution= 0.00025, lf_resolution= 0.002)

In [ ]:
# End-based synchronization (just for experiment 5)
# a= 516819
# b= 600100
# shift= b-a
# distance= hf_laser_sync['distance'].copy()
# distance= distance.shift(periods= -shift, axis= 0, fill_value= 0)
# print(distance.shape)
# hf_laser_sync['distance']= distance
# hf_laser_sync= hf_laser_sync.iloc[: hf_laser_sync.shape[0]-shift]

In [ ]:
####Visualization of Hf- Laser synchronization results
selected_columns= ['ENC1_POS|8','is_speed',  'is_acceleration', 'POWER|8',  'CURRENT|8',  'ENC2_POS|3', 'distance']
y_axis_names= ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)', 'distance']
fig= make_subplots(rows=len(selected_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
for i in range(len(selected_columns)):
    fig.add_trace(go.Line(y= hf_laser_sync[selected_columns[i]]), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
fig.update_layout(height=1200, width=1200, title_text="HF-Data")
fig.show()

In [ ]:
# #### Read LF data ##old data structure
# # lf_path= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\Spindlewachstumsversuch\\220414_02\\lf_data.json'
# event_list = db.lf_data.find().sort('date', 1)
# lf_signals= None
# for event in event_list:
#     keys= event['content']['raw_data'].keys()
#     for item in keys:
#         event['content']['raw_data'][item]= [event['content']['raw_data'][item]]
#     event['content']['raw_data']['date']= event['date']
#     event['content']['raw_data']['DRZ'][0]= float(event['content']['raw_data']['DRZ'][0])
#     lf_signal= pd.DataFrame(event['content']['raw_data'])
#     if lf_signals is None:
#         lf_signals= lf_signal
#     else:
#         lf_signals= lf_signals.append(lf_signal,ignore_index= True)
# lf_signals['t_motor']= lf_signals['t_motor'].apply(np.float32)
# print (lf_signals.head(3))


In [ ]:
#### Read LF data ##New data structure
event_list = db.lf_data.find().sort('date', 1)
lf_signals= None
for event in event_list:
    record_list= event['content']
    keys= None
    for record in record_list:
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        record['raw_data']['date']=[record['date']]
        record['raw_data']['DRZ'][0]= float(record['raw_data']['DRZ'][0])
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if lf_signals is None:
            lf_signals= lf_signal_point
        else:
            lf_signals= lf_signals.append(lf_signal_point,ignore_index= True)
lf_signals['t_motor']= lf_signals['t_motor'].apply(np.float32)
lf_signals.reset_index(inplace= True)
print (lf_signals.info())

In [ ]:
#### Read LF data ## with predictions
event_list = db.lf_data.find().sort('date', 1)
lf_signals= None
for event in event_list:
    record_list= event['content']
    keys= None
    for record in record_list:
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        record['raw_data']['date']=[record['date']]
        record['raw_data']['DRZ'][0]= float(record['raw_data']['DRZ'][0])
        record['raw_data']['prediction']=[record['prediction']]
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if lf_signals is None:
            lf_signals= lf_signal_point
        else:
            lf_signals= lf_signals.append(lf_signal_point,ignore_index= True)
lf_signals['t_motor']= lf_signals['t_motor'].apply(np.float32)
lf_signals.reset_index(inplace= True)
print (lf_signals.info())

In [ ]:
## Visualization of lf-data with predicted results
selected_columns= ['t_spindle', 'DRZ', 't_motor','t_welle', 'prediction']
y_axis_names= selected_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
fig= make_subplots(rows=len(selected_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
for i in range(len(selected_columns)):
    fig.add_trace(go.Line(y= lf_signals[selected_columns[i]]), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
fig.update_layout(height=1800, width=1200, title_text="LF-Data")
fig.show()

In [ ]:
sync_point_lf= 0#2585
lf_path= str(experiment_path+'sync_result\\sync_220429_03.csv')
lf_df= pd.read_csv(lf_path)
lf_df['prediction']= lf_signals['prediction'][sync_point_lf:]
lf_df['displacement']= (lf_df['distance'][0] - lf_df['distance'])#*10000
#result_df['displacement']= result_df['displacement'].mask(result_df['displacement']<0, 0)
lf_df['rolling_mean_displacement']= lf_df['displacement'].rolling(20).mean()

In [ ]:
selected_columns= ['t_spindle', 'DRZ', 't_motor','t_welle', 'displacement', 'rolling_mean_displacement','distance']
y_axis_names= selected_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
fig= make_subplots(rows=len(selected_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
for i in range(len(selected_columns)):
    fig.add_trace(go.Line(y= lf_df[selected_columns[i]]), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
fig.update_layout(height=1800, width=1200, title_text="LF-Data")
fig.show()

In [ ]:
result_df= synchronize(df1=hf_laser_sync, df2= lf_signals, rule= {'df1':('index',569), 'df2':('index',326)})

In [ ]:
# print(result_df.info())
# selected_columns= ['ENC1_POS|8','CMD_SPEED|8','is_speed',  'is_acceleration', 'POWER|8',  'CURRENT|8',  'ENC2_POS|3','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
# y_axis_names= selected_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
# fig= make_subplots(rows=len(selected_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
# for i in range(len(selected_columns)):
#     fig.add_trace(go.Line(y= result_df[selected_columns[i]]), row= i+1, col= 1)
#     fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
# fig.update_layout(height=1200, width=1200, title_text="LF-Data")
# fig.show()

In [ ]:
def calculate_temperature(df: pd.DataFrame, l= 1, alpha=1):
    result= []
    l1= l
    l2=l1
    for i in range(df.shape[0]):
        l2= df.iloc[i]/1000
        delta_l= l2 - l1
        delta_t= delta_l/(alpha * l1)
        l1= l2
        result.append(delta_t)
    return result
    

In [ ]:
def calculate_displacement(df: pd.DataFrame, t= 1, alpha=1):
    result= []
    l1=1
    t1= t
    t2=t1
    for i in range(df.shape[0]):
        t2= df.iloc[i]
        delta_t= t2 - t1
        delta_l= l1*alpha* delta_t
        t1= t2
        result.append(delta_l)
    return result

In [ ]:
result_df['displacement']= (result_df['distance'][0] - result_df['distance'])#*10000
#result_df['displacement']= result_df['displacement'].mask(result_df['displacement']<0, 0)
result_df['rolling_mean_displacement']= result_df['displacement'].rolling(20).mean()
result_df['delta_t']= calculate_temperature(result_df['rolling_mean_displacement'], alpha= 12)
result_df['delta_t']= result_df['delta_t'].fillna(0)
#result_df['rolling_mean_displacement']= result_df['displacement'].rolling(100).mean()#['rolling_mean_displacement']#
res= [item for item in it.accumulate(result_df['delta_t'].to_list())]
#print('result= ', res)
result_df['delta_t_acc']= res
###### Calculate displacement
result_df['displacement']= result_df['displacement']
result_df['displacement']= result_df['displacement'].mask(result_df['displacement']<0, 0)
result_df['rolling_mean_displacement']= result_df['displacement'].rolling(20).mean()
displacement= calculate_displacement(result_df['t_welle'], alpha= 12)
displacement= [item/1000 for item in it.accumulate(displacement)]
result_df['calcuated_displacement']= displacement

In [ ]:
#selected_columns= ['ENC1_POS|8','is_speed',  'is_acceleration', 'POWER|8',  'CURRENT|8',  'ENC2_POS|3']
#selected_columns= ['ENC1_POS|8','is_speed',  'is_acceleration', 'POWER|8',  'CURRENT|8',  'ENC2_POS|3','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 't_welle','distance', 'displacement', 'rolling_mean_displacement']
selected_columns= ['ENC1_POS|8','is_speed','is_acceleration','POWER|8',  'CURRENT|8', 't_spindle', 'DRZ', 't_motor','t_welle','delta_t_acc', 'displacement', 'rolling_mean_displacement', 'calcuated_displacement']
y_axis_names= selected_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
fig= make_subplots(rows=len(selected_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
for i in range(len(selected_columns)):
    fig.add_trace(go.Line(y= result_df[selected_columns[i]]), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
fig.update_layout(height=1800, width=1200, title_text="LF-Data")
fig.show()
#fig.write_image(str(experiment_path + 'sync_result\\sync_results.png'))

In [ ]:
#[781:858]
#[1106:1183]
#[1265:1342]
series= result_df['displacement'][1106:1183]
rolling= series.rolling(10).mean()


In [ ]:
fig= make_subplots(rows= 2 ,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_columns, vertical_spacing=0.02)
fig.add_trace(go.Line(y= series), row= 1, col= 1)
fig.update_yaxes(title_text='orginal', row= 1, col= 1)
fig.add_trace(go.Line(y= rolling), row= 2, col= 1)
fig.update_yaxes(title_text='rolling', row= 2, col= 1)
fig.update_layout(height=1800, width=1200, title_text="LF-Data")
fig.show()

In [ ]:
#### calculate the effect of centrifugal Force
up= 10000.0
down= 0.0
down_index= -1
up_index= -1
window= 5
result_list= []
for index, row in result_df.iterrows():
    drhz= np.round(row['DRZ'], -3)
    #print(drhz)
    if drhz == up:
        up_index= index
        down_index= -1
    if drhz == down:
        down_index= index
        if up_index != -1:
            print('up_index= ', up_index, ', down_index= ', down_index)
            cf_effect= result_df['displacement'][down_index: down_index + window].median()
            result_list.append({'up_index': up_index,'down_index': down_index,'cf_effect': cf_effect})
        up_index= -1
print(result_list)




In [ ]:
# selected_columns= ['date','ENC1_POS|8','is_speed',  'is_acceleration', 'POWER|8',  'CURRENT|8',  'ENC2_POS|3','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett','t_welle', 'distance']
# lf_path= str(experiment_path+'sync_result\\sync_220429_05.csv')
# result_df[selected_columns].to_csv(lf_path)